# Install libs

In [ ]:
!pip install nltk spacy scikit-learn gensim matplotlib seaborn pandas tqdm flair

# Wed morning (NN on word frequency)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib as mp
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
%load_ext tensorboard

## 1. Data: inspection and loading

First - open the file and check it. e.g. in a notepad++

In [ ]:
cp "/content/drive/MyDrive/NLP_M3_data/spambase.csv" .

In [ ]:
# cat 'spambase.csv'

In [ ]:
df = pd.read_csv('spambase.csv')

  (a) Creators: Mark Hopkins, Erik Reeber, George Forman, Jaap Suermondt
      Hewlett-Packard Labs, 1501 Page Mill Rd., Palo Alto, CA 94304

  (b) Donor: George Forman (gforman at nospam hpl.hp.com)  650-857-7835
  
  (c) Generated: June-July 1999


In [ ]:
df

This table contains information about email texts:

The last column of 'spambase.data' denotes whether the e-mail was
considered spam (1) or not (0), i.e. unsolicited commercial e-mail.  
Most of the attributes indicate whether a particular word or
character was frequently occuring in the e-mail.  The run-length
attributes (55-57) measure the length of sequences of consecutive
capital letters.  For the statistical measures of each attribute,
see the end of this file.  Here are the definitions of the attributes:

48 continuous real [0,100] attributes of type word_freq_WORD
= percentage of words in the e-mail that match WORD,
i.e. 100 * (number of times the WORD appears in the e-mail) /
total number of words in e-mail.  A "word" in this case is any
string of alphanumeric characters bounded by non-alphanumeric
characters or end-of-string.

6 continuous real [0,100] attributes of type char_freq_CHAR
= percentage of characters in the e-mail that match CHAR,
i.e. 100 * (number of CHAR occurences) / total characters in e-mail

1 continuous real [1,...] attribute of type capital_run_length_average
= average length of uninterrupted sequences of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_longest
= length of longest uninterrupted sequence of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_total
= sum of length of uninterrupted sequences of capital letters
= total number of capital letters in the e-mail

1 nominal {0,1} class attribute of type spam
= denotes whether the e-mail was considered spam (1) or not (0),
i.e. unsolicited commercial e-mail.  




In [ ]:
sum(df.spam==1), sum(df.spam==0)

Overall variable statistics:

In [ ]:
df.describe().T

Always check if your data has missing values:

In [ ]:
df.isna().max()

## 2. Data preparation

We will first to split the table into independent variables, and target variable.

In [ ]:
x = df.drop('spam', axis=1).values.astype(np.float32)
y = df['spam'].values.astype(np.int64)
column_names = list(df.columns)

In [ ]:
x.shape, x.dtype

In [ ]:
y.shape, y.dtype

Let's understand the input feature distributions.
We could make a histogram for each value, but this is exactly what violin plot does for multiple variables.

e.g. for first two columns:

In [ ]:
column_names[:2]

In [ ]:
plt.hist(x[:, 0], 100);

In [ ]:
n_col = 1
df_x = pd.DataFrame(x[:, :n_col], columns=column_names[:n_col])

plt.figure(figsize=(15, 4)) # always make plots big enough so you see there anything
sns.violinplot(data=df_x, cut=0)

plt.xticks(rotation=45, horizontalalignment='right'); # make text visible: @ 45 degree + aligned rihght

now for all colums:

In [ ]:
df_x = pd.DataFrame(x, columns=column_names[:-1])

plt.figure(figsize=(15, 4)) # always make plots big enough so you see there anything
sns.violinplot(data=df_x, cut=0)

plt.xticks(rotation=45, horizontalalignment='right'); # make text visible: @ 45 degree + aligned rihght

We see that range of values is very different for different columns. This can negatively impact the training of neural networs.

It is always a good practice to standardise inputs (and continous outputs)

To make distribution of all features similar - we standardize them:

In [ ]:
x_s = StandardScaler().fit_transform(x)

In [ ]:
df_x = pd.DataFrame(x_s, columns=column_names[:-1])

plt.figure(figsize=(25, 4))
sns.violinplot(data=df_x, cut=0)

plt.xticks(rotation=45, horizontalalignment='right');

Finally to be able to chech the performace of out models - we will split data into independent training and validation sets:

In [ ]:
x_train, x_validation, y_train, y_validation = train_test_split(x_s, y, shuffle=True, stratify=y, test_size=0.2)

In [ ]:
x_train.shape, x_validation.shape, y_train.shape, y_validation.shape

## 3. Data loading

We will perform model trainign on the GPU if it is available. This means we will:
1. place model, inputs and trget data on the `device`
2. afterwards move the resulting calculations performed on device back to 'host' - memory directly accessible by CPU

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')  # use first available GPU
    else:
        device = torch.device('cpu')
    return device

In [ ]:
device = get_device()

Convert numpy arrays into torch tensors

In [ ]:
x_tra_tensor =  torch.tensor(x_train).to(device)
y_tra_tensor =  torch.tensor(y_train).to(device)
x_val_tensor =  torch.tensor(x_validation).to(device)
y_val_tensor =  torch.tensor(y_validation).to(device)

In [ ]:
x_train

In [ ]:
x_tra_tensor

for model training - we will group the independent variables (model input) ad the dependent (target, desired model ouput) together

In [ ]:
tra_tensor = data_utils.TensorDataset(x_tra_tensor, y_tra_tensor)
val_tensor = data_utils.TensorDataset(x_val_tensor, y_val_tensor)

In [ ]:
tra_tensor

In [ ]:
tra_tensor.tensors

The stochastic gradient descent optimizations are performed sequentially on parts of the training algoritms - (mini)batches.

In [ ]:
BATCH_SIZE = 100

`DataLoader` is a utility class that helps with efficiently loading and iterating over large datasets for the training:

In [ ]:
tra_loader = data_utils.DataLoader(dataset=tra_tensor, batch_size=BATCH_SIZE,
                                   shuffle=True, drop_last=True,
                                   #num_workers=2, persistent_workers=True  # <- good for produciton
                                   )

val_loader = data_utils.DataLoader(dataset=val_tensor, batch_size=BATCH_SIZE,
                                   shuffle=False, drop_last=False,
                                   #num_workers=2, persistent_workers=True  # <- good for produciton
                                   )

In [ ]:
for i, s in enumerate(tra_loader):
  print(i, s)

In [ ]:
for i, s in enumerate(tra_loader):
  print(i, s[0].shape, s[1].shape)

In [ ]:
print('training size after batching:', len(tra_loader) * tra_loader.batch_size)
print('training dataset size:', len(x_tra_tensor))

## 4. Model

When designing your model first thing to think about - is waht are you putting in and what do you want out.
Think of it in term of:
- shape
- type
- value range

these would define:
- ?
- ?

In [ ]:
x_s.shape, y.shape

In [ ]:
class SpamClassificationModel_1(nn.Module):
  def __init__(self):
    super(SpamClassificationModel_1, self).__init__()

    # single layer model
    self.l1 = nn.Linear(57, 2)

  def forward(self, x):
    y = x
    #print('input: ', y.shape)

    # output - just logits. for optimized trainign - without activaiton here
    y = self.l1(y)  # returns logits

    return y

In [ ]:
model_scm1 = SpamClassificationModel_1().to(device)

In [ ]:
x_val_tensor.shape

In [ ]:
# forward pass of validation set through the model, to see output distribution
pred = model_scm1(x_val_tensor)

In [ ]:
pred.shape

In [ ]:
pred_proba = F.softmax(pred.detach(), dim=1)

# to get value from a tensor - we copy it to CPU-accessible memory,
# then convert to a numpy array
pred_proba_value = pred_proba.cpu().numpy()

In [ ]:
pred_proba_value.shape

In [ ]:
# brobability belonging to class 1 - spam:
class1_proba_value = pred_proba_value[:, 1]

# plot histogram
plt.hist(class1_proba_value, 100);

In [ ]:
plt.scatter(y_validation, class1_proba_value)

## 5. Trainig

In [ ]:
# Define the loss function (Cross Entropy Loss)
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer (Stochastic Gradient Descent)
optimizer = optim.SGD(model_scm1.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Number of epochs for training
N_EPOCHS = 1000

In [ ]:
# Initialize empty lists to store training and validation loss and accuracy
tra_loss = []
val_loss = []
tra_acc = []
val_acc = []

# Calculate the total number of training and validation samples
n_tra = len(tra_loader) * tra_loader.batch_size
n_val = len(x_validation)

# Iterate over epochs
for epoch in tqdm.tqdm_notebook(range(N_EPOCHS)):  # loop over the dataset `N_EPOCHS` times
    model_scm1.train()  # Set the model to training mode
    # 1. Training
    epoch_loss = 0
    epoch_correct = 0
    for i, data in enumerate(tra_loader):

        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model_scm1(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_correct += torch.sum(torch.argmax(outputs, dim=1) == labels).detach().cpu().numpy()

    tra_loss.append(epoch_loss / (i + 1))
    tra_acc.append(epoch_correct / n_tra)

    model_scm1.eval()  # Set the model to evaluation mode

    # 2. Validation
    epoch_loss = 0
    epoch_correct = 0
    for i, data in enumerate(val_loader):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        with torch.no_grad(): # disable gradient calculation during evaluation
            outputs = model_scm1(inputs)
            loss = loss_fn(outputs, labels)

            epoch_loss += loss.item()
            epoch_correct += torch.sum(torch.argmax(outputs, dim=1) == labels).detach().cpu().numpy()

    val_loss.append(epoch_loss / (i + 1))
    val_acc.append(epoch_correct / n_val)

print('Finished Training')


In [ ]:
# plot loss and accuracy evolution
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(tra_loss, label='training loss')
ax[0].plot(val_loss, label='validation loss')
ax[0].legend()

ax[1].plot(tra_acc, label='training accuracy')
ax[1].plot(val_acc, label='validation accuracy')
ax[1].legend()

plt.show()
plt.close()


What do you see?

lets inspect the model's performance differently

In [ ]:
pred = model_scm1(x_val_tensor)

In [ ]:
pred_value = F.softmax(pred.detach(), dim=1).cpu().numpy()

In [ ]:
plt.hist(pred_value[:, 1], 100);

In [ ]:
cm = confusion_matrix(y_validation, pred_value.argmax(axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

What do you see?

## 6. Model inspection

In this section we will make use of the Tensorboard to visualize the training evolution and the model's architecture

In [ ]:
#rm -fr runs

In [ ]:
# Create a SummaryWriter for TensorBoard logging, specifying the directory 'runs/model_scm1' for storing logs
writer = SummaryWriter('runs/model_scm1')

# Define a layout for custom scalars in TensorBoard
layout = {
    "LA": {
        "loss": ["Multiline", ["loss/training", "loss/validation"]],
        "accuracy": ["Multiline", ["accuracy/training", "accuracy/validation"]],
    },
}

# Add the custom scalars layout to TensorBoard
writer.add_custom_scalars(layout)

# Create an instance of the SpamClassificationModel_1 and move it to the selected device (e.g., GPU if available)
model_scm1 = SpamClassificationModel_1().to(device)

# Get a batch of data samples and labels from the training DataLoader
xs, ys = next(iter(tra_loader))

# Add the model's computation graph to TensorBoard, visualizing the model's architecture
writer.add_graph(model_scm1, xs)

In [ ]:
# Define the loss function (Cross Entropy Loss)
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer (Adam optimizer)
# - model_scm1.parameters(): Specifies the model's parameters that need optimization
# - lr=0.001: Learning rate determines the step size for parameter updates during training
optimizer = optim.Adam(model_scm1.parameters(), lr=0.001)

In [ ]:
N_EPOCHS = 2000

In [ ]:
# Calculate the total number of training and validation samples
n_tra = len(tra_loader) * tra_loader.batch_size  # Total training samples
n_val = len(x_validation)  # Total validation samples

# Iterate over the specified number of training epochs
for epoch in tqdm.tqdm_notebook(range(N_EPOCHS)):  # Loop over the dataset multiple times

    # Set the model to training mode
    model_scm1.train()

    # 1. Training
    epoch_loss = 0
    epoch_correct = 0

    # Iterate over batches of training data
    for i, data in enumerate(tra_loader):

        # Get the inputs and labels from the batch; data is a list of [inputs, labels]
        inputs, labels = data

        # Zero the gradients of the model parameters (reset gradients)
        optimizer.zero_grad()

        # Forward pass: Compute model predictions for the inputs
        outputs = model_scm1(inputs)

        # Compute the loss between model predictions and actual labels
        loss = loss_fn(outputs, labels)

        # Backward pass: Compute gradients of the loss with respect to model parameters
        loss.backward()

        # Update model parameters using the optimizer (perform one optimization step)
        optimizer.step()

        # Accumulate the loss and calculate the number of correctly classified samples in this batch
        epoch_loss += loss.item()
        epoch_correct += torch.sum(torch.argmax(outputs, dim=1) == labels).detach().cpu().numpy()

    # Log training loss and accuracy for this epoch using TensorBoard
    writer.add_scalar('loss/training', epoch_loss / (i + 1), epoch)
    writer.add_scalar('accuracy/training', epoch_correct / n_tra, epoch)

    # Set the model to evaluation mode
    model_scm1.eval()

    # 2. Validation
    epoch_loss = 0
    epoch_correct = 0

    # Iterate over batches of validation data
    for i, data in enumerate(val_loader):

        # Get the inputs and labels from the batch; data is a list of [inputs, labels]
        inputs, labels = data

        # Perform forward pass without gradient computation (no gradient updates)
        with torch.no_grad():
            outputs = model_scm1(inputs)
            loss = loss_fn(outputs, labels)

            # Accumulate the loss and calculate the number of correctly classified samples in this batch
            epoch_loss += loss.item()
            epoch_correct += torch.sum(torch.argmax(outputs, dim=1) == labels).detach().cpu().numpy()

    # Log validation loss and accuracy for this epoch using TensorBoard
    writer.add_scalar('loss/validation', epoch_loss / (i + 1), epoch)
    writer.add_scalar('accuracy/validation', epoch_correct / n_val, epoch)

# Training loop is finished
print('Finished Training')

# Close the TensorBoard writer to save the logs
writer.close()


run tensorboard to visualize data:

In [ ]:
%tensorboard --logdir=runs

## 7. Exercise 1.

Use model with 2 layers
Evaluate effect of the hyperparameters on model performance:
- learnign rate (1.e-5, 3.e-5, 1.e-4, 3.e-4, 1.e-3, 3.e-3, 1.e-3, 3.e-3, 1.e-2, 3.e-2, 1.e-1, 3.e-1)
- batch size (2, 4, 8, 16, 32, 64, 128, 256, 512)

fill the table from several runs and plot the statistics. divide in groups to work on one task.
Dicuss in your group additional metrics you could evaluate and add them to the table.

In [ ]:
results = {
    'lr': [],
    'batch_size': [],
    'best_val_accuracy': [],
    'best_val_acc_epoch': []
}

In [ ]:
# plot runs comparisson. e.g. for the learning rate:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(results['lr'], results['best_val_accuracy'], '*')

ax[1].plot(results['lr'], results['best_val_acc_epoch'], '*')

plt.show()
plt.close()


## 8. Exercise 2.

Explore different model architectures: number of layers, number of features (neurons) per layer, etc

Fill a dictionaly sililar to exercise 1, and compare the results of different models.

# Wed evening (NN on tf-idf)

In this session we will continue with building simple neural networks.
We will use the more sophisticated features, and rely on previously established intuition about building the NNs.

## 1. Imports

In [ ]:
# Import the necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt

## 2. Fetch dataset

In [ ]:
fetch_20newsgroups().target_names

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). The split between the train and test set is based upon a messages posted before and after a specific date.

We strip the headers and footers, as those can make the task easier.

In [ ]:
# Load the 20 Newsgroups dataset
categories = ['rec.sport.hockey', 'sci.electronics', 'comp.graphics']
SEED = 64
train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=SEED, remove=('headers', 'footers', 'quotes'))
test_data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=SEED, remove=('headers', 'footers', 'quotes'))

In [ ]:
# inspect the data structure
train_data.keys()

In [ ]:
# inspect the data input elements
train_data.data[:3]

## 3. Preprocess and inspect dataset

In [ ]:
# Iterate over the train_data and test_data
for d in [train_data, test_data]:

  # Remove leading and trailing whitespace, tab,
  # and new line characters from each data point
  d.data = [s.strip(' \n\t\r') for s in d.data]

  # Get the indices of data points with non-empty content
  ok_idx = [i for i, s in enumerate(d.data) if len(s) > 0]

  # Filter the data and target lists to keep only non-empty data points
  d.data = [d.data[i] for i in ok_idx]
  d.target = [d.target[i] for i in ok_idx]

In [ ]:
# inspect the data labels elements
train_data.target

In [ ]:
len(train_data.data), len(test_data.data)

In [ ]:
sample = train_data.data[0]
sample_label = train_data.target[0]


In [ ]:
print(sample)
print(f'label={sample_label}, ({categories[sample_label]})')

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
# Convert the text data to TF-IDF vectors, max 10 words
vectorizer = TfidfVectorizer(max_features=10)

# `object.fit` followed by processing by `object.transform` or the
# joined `object.fit_transform` is the
train_vectors = vectorizer.fit_transform(train_data.data[:3])

In [ ]:
feature_names = vectorizer.get_feature_names_out()
print(len(feature_names))

In [ ]:
# the vectors are stored in sparse format:
train_vectors

In [ ]:
# to get the dense representation we need to convert them to an array:
train_vectors.toarray()

In [ ]:
plt.figure(figsize=(30, 3))
plt.plot(vectorizer.get_feature_names_out(), train_vectors.toarray()[0])
plt.plot(vectorizer.get_feature_names_out(), train_vectors.toarray()[1])
plt.plot(vectorizer.get_feature_names_out(), train_vectors.toarray()[2])
plt.xticks(rotation=45, horizontalalignment='right');
plt.ylim(0,1);

In [ ]:
# Convert the text data to TF-IDF vectors, all words
vectorizer = TfidfVectorizer(max_features=None)
train_vectors = vectorizer.fit_transform(train_data.data)
test_vectors = vectorizer.transform(test_data.data)

In [ ]:
af = train_vectors.toarray().flatten()
# plot only present words (tfidf>0)
aff = af[af>0]
plt.hist(aff, 100);

## 4. Create and train the model

In [ ]:
# Define the model
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Define the model hyperparameters
input_dim = train_vectors.shape[1]
hidden_dim = 256
output_dim = len(categories)

# Create the model instance
model = Classifier(input_dim, hidden_dim, output_dim)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# Set the device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model and data to the device
model = model.to(device)
train_vectors_t = torch.tensor(train_vectors.toarray(), dtype=torch.float).to(device)
train_labels_t = torch.tensor(train_data.target, dtype=torch.long).to(device)
test_vectors_t = torch.tensor(test_vectors.toarray(), dtype=torch.float).to(device)
test_labels_t = torch.tensor(test_data.target, dtype=torch.long).to(device)

# Training loop
def train(model, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    predictions = model(train_vectors_t)
    loss = criterion(predictions, train_labels_t)
    loss.backward()
    optimizer.step()

# Evaluation loop
def evaluate(model):
    model.eval()
    with torch.no_grad():
        predictions = model(test_vectors_t)
        predicted_labels = torch.argmax(predictions, dim=1)
        accuracy = torch.sum(predicted_labels == test_labels_t).item() / len(test_labels_t)
    return accuracy

# Training and evaluation loop
NUM_EPOCHS = 100
accuracy_arr = []
for epoch in tqdm(range(NUM_EPOCHS)):
    train(model, optimizer, criterion)
    accuracy = evaluate(model)
    #print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Accuracy: {accuracy:.4f}")

    accuracy_arr.append(accuracy)

plt.plot(accuracy_arr)

## 5. Exercise

Work in 3 groups:
 - present final group results in the end (5 min per group):
 - half time through - share code of your intermediate results (on Zoom) so that other groups use it for final results:

1. Optimize the model architecture to improve performance.
2. Study dependence of performance & training time on number of tf-idf features
3. Study relevant performance metrics. Make evaluation code for both training and the test sets.

# Thu morning RNN on embeddings

## 1. Imports/utils

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as gd
from sklearn.datasets import fetch_20newsgroups

import re

In [ ]:
import nltk
import tqdm
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nltk.download('punkt')

In [ ]:
def load_pckl(file_name, path=None):
    if path is not None:
        file_name = os.path.join(path, file_name)

    with open(file_name, 'rb') as f:
        data = pickle.load(f)
    return data


def save_pckl(d, file_name, pr=None, path=None):
    if path is not None:
        file_name = os.path.join(path, file_name)

    with open(file_name, 'wb') as f:
        pickle.dump(d, f, protocol=pr if pr is not None else pickle.DEFAULT_PROTOCOL)

In [ ]:
def remove_punctuation(input_string):
    # Use regular expression to remove all punctuation characters
    return re.sub(r'[^\w\s]', '', input_string)  # everything which is not a word (\w) or space (\s) -> empty string ('')

## 2. Load and preprocess the dataset

In [ ]:
# Load the 20 Newsgroups dataset
categories = ['rec.sport.hockey', 'sci.electronics', 'comp.graphics']
SEED = 64
train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=SEED, remove=('headers', 'footers', 'quotes'))
test_data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=SEED, remove=('headers', 'footers', 'quotes'))

In [ ]:
# Iterate over the train_data and test_data
for d in [train_data, test_data]:

  # Remove leading and trailing whitespace, tab,
  # and new line characters from each data point
  d.data = [s.strip(' \n\t\r') for s in d.data]

  # Get the indices of data points with non-empty content
  ok_idx = [i for i, s in enumerate(d.data) if len(s) > 0]

  # Filter the data and target lists to keep only non-empty data points
  d.data = [d.data[i] for i in ok_idx]
  d.target = [d.target[i] for i in ok_idx]

## 3. Explore Word2Vec (Homework)

Similar to the tiny dataset example explore similar words and word arythmetic.

Use your imagination, e.g. snake-long+short, or brick-hard+soft, etc

### 1. Tiny dataset

In [ ]:
# Preprocess the corpus (replace with your own preprocessing steps)
corpus = ["I love to eat pizza", "I hate Mondays", "Pizza is delicious", "I enjoy playing tennis"]

# Tokenize the corpus
tokenized_corpus = [sentence.lower().split() for sentence in corpus]

# Train Word2Vec model
model = Word2Vec(tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4, )

In [ ]:
# Find similar words
similar_words = model.wv.most_similar("pizza")
print("Similar words to 'pizza':")
for word, similarity in similar_words:
    print(word, similarity)

In [ ]:
# Perform word embeddings arithmetic
result = model.wv.most_similar(positive=["tennis", "hate"], negative=["love"])  # "tenis" - "love" + "hate"
print("Word embeddings arithmetic:")
for word, similarity in result:
    print(word, similarity)


As you see - on tiny dataset the vectors don't make much sense.
You are encouraged to explore the embedding vectors built based on the bigger datasets in the nxt 2 sections

### 2. Bigger dataset

In [ ]:
# Preprocess the corpus (replace with your own preprocessing steps)
corpus = train_data.data

# Tokenize the corpus
tokenized_corpus = [remove_punctuation(sentence.lower()).split() for sentence in corpus]

# Train Word2Vec model
model = Word2Vec(tokenized_corpus, vector_size=300, window=5, min_count=1, workers=4, )

In [ ]:
word = "network"
similar_words = model.wv.most_similar(word, topn=20)
print(f"Similar words to '{word}':")
for word, similarity in similar_words:
    print(word, similarity)

Try removing the stopwords and re-fitting the model (see sections below hoe to do it)



### 3. 300k words

Here a pretrained model on many news articles is downloaded (Downloading it can take a while!). Feature vectors are of length 300, total 300k words.

In [ ]:
# Download the word2vec-google-news-300 model
model_gn300 = gd.load('word2vec-google-news-300')

In [ ]:
model_gn300.vectors.shape

In [ ]:
word = "car"
similar_words = model_gn300.most_similar(word)
print(f"Similar words to '{word}':")
for word, similarity in similar_words:
    print(word, similarity)

In [ ]:
word = "god"
similar_words = model_gn300.most_similar(word)
print(f"Similar words to '{word}':")
for word, similarity in similar_words:
    print(word, similarity)

In [ ]:


# Find similar words
similar_words = model_gn300.most_similar("python")
print("Similar words to 'pizza':")
for word, similarity in similar_words:
    print(word, similarity)

# Perform word embeddings arithmetic
result = model_gn300.most_similar(positive=["python", "young"], negative=["old"])
print("Word embeddings arithmetic:")
for word, similarity in result:
    print(word, similarity)

# Visualize word embeddings using t-SNE (replace with your own visualization code)
# ...

# Additional tasks and experiments with word embeddings
# ...


## 4. Preparing embedding dataset (not run in the class - time-consuming)

### 0. Embedding utilities

First we create word embeddigns with Word2Vec

In [ ]:
# Download (takes a while!) the word2vec-google-news-300 model
model_gn300 = gd.load('word2vec-google-news-300')

In [ ]:
def get_vec(model, word):
  # convert word to vector
  # Check if the model has an index for the word, of so get vector,
  # otherwise return None
  return model.get_vector(word) if model.has_index_for(word) else None

In [ ]:
# Function to convert text to word vectors
# using a specified word embedding model
# (default - word2vec-google-news-300 model, Download takes a while!
# you can also try yours)
def convert_text_to_vecs(text, model=model_gn300):
  # Tokenize the input text into individual words
  words = nltk.tokenize.word_tokenize(text)

  # Get word vectors for each word in the text using the specified word embedding model
  wvs = [get_vec(model, word) for word in words]

  # Filter out None values (word vectors that couldn't be found in the model)
  wvs = [v for v in wvs if v is not None]

  # Convert the list of word vectors to a NumPy array
  wvs = np.array(wvs)

  # Return the array of word vectors for the input text
  return wvs

Few helper functions to convert input dataset, and to save whole dataset to file

In [ ]:
def convert_corpus_to_vecs(corpus, convert_corpus_to_vecs_fn):
  # for each text in the corpus - vectorize it
  # using list comprehension. Use tqdm to display progress
  return [convert_corpus_to_vecs_fn(text) for text in tqdm.auto.tqdm(corpus)]

In [ ]:
def prepare_embedding_dataset(tra_input, tra_labels,
                              val_input, val_labels,
                              filename,
                              convert_text_to_vecs_fn):
  # Prepare and save an embedding dataset for training and validation.
  print('embedding training data:')
  tra_data_vecs = convert_corpus_to_vecs(tra_input, convert_text_to_vecs_fn)

  print('embedding validation data:')
  val_data_vecs = convert_corpus_to_vecs(val_input, convert_text_to_vecs_fn)

  # remove empty elements
  ok_idx = [i for i, s in enumerate(tra_data_vecs) if len(s)>0]
  tra_data_vecs = [tra_data_vecs[i] for i in ok_idx]
  tra_labels = [tra_labels[i] for i in ok_idx]

  ok_idx = [i for i, s in enumerate(val_data_vecs) if len(s)>0]
  val_data_vecs = [val_data_vecs[i] for i in ok_idx]
  val_labels = [val_labels[i] for i in ok_idx]

  print(f'preparing and saving dataset to: {filename}')

  # Create a dataset dictionary containing training and validation data and labels
  dataset = {
    'tra_data': tra_data_vecs,
    'tra_labels': tra_labels,
    'val_data': val_data_vecs,
    'val_labels': val_labels,
  }

  # Save the dataset as a pickle file
  save_pckl(dataset, filename, path='./')

GloVe and RoBERTa embeddings (with flair)

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings, TransformerWordEmbeddings
from flair.data import Sentence

In [ ]:
# Download models for word embeddings with GloVe and RoBERTa
glove_embedding = WordEmbeddings('glove')
roberta_embedding = TransformerWordEmbeddings('roberta-base')

# Create a DocumentPoolEmbeddings object (optional but helpful)
document_embeddings_glove = DocumentPoolEmbeddings([glove_embedding])
document_embeddings_roberta = DocumentPoolEmbeddings([roberta_embedding])


In [ ]:
def convert_text_to_embedding(text, doc_embedding_model):
  # Create a Flair Sentence object
  sentence = Sentence(text)

  # Embed the sentence
  try:
    doc_embedding_model.embed(sentence)
  except Exception as e:
    # if model can't convert a text - print it, before raising the exception
    print('failed text:', text)
    raise e

  embeddings = [token.embedding.cpu().numpy() for token in sentence]
  return np.array(embeddings)

In [ ]:
def convert_text_to_glove(text):
  return convert_text_to_embedding(text, document_embeddings_glove)

def convert_text_to_roberta(text):
  return convert_text_to_embedding(text, document_embeddings_roberta)


### 1. Create word2vec embeddings

In [ ]:
prepare_embedding_dataset(train_data.data, train_data.target,
                          test_data.data, test_data.target,
                          filename='dataset_20newsgroups_3_cat.pckl',
                          convert_text_to_vecs_fn=convert_text_to_vecs
                          )

In [ ]:
!mv dataset_20newsgroups_3_cat.pckl "/content/drive/MyDrive/Colab Data/NLP_M3_data"

### 2. Create GloVe embeddings

In [ ]:
prepare_embedding_dataset(train_data.data, train_data.target,
                          test_data.data, test_data.target,
                          filename='dataset_20newsgroups_3_cat_gl.pckl',
                          convert_text_to_vecs_fn=convert_text_to_glove
                          )

In [ ]:
!mv dataset_20newsgroups_3_cat_gl.pckl "/content/drive/MyDrive/Colab Data/NLP_M3_data"

### 3. Create RoBERTa embeddings (run with a GPU)

In [ ]:
prepare_embedding_dataset(train_data.data, train_data.target,
                          test_data.data, test_data.target,
                          filename='dataset_20newsgroups_3_cat_rb.pckl',
                          convert_text_to_vecs_fn=convert_text_to_roberta
                          )

In [ ]:
!mv dataset_20newsgroups_3_cat_rb.pckl "/content/drive/MyDrive/Colab Data/NLP_M3_data"

### 4. Inspect prepared datasets:

In [ ]:
dataset_20newsgroups_3_cat = load_pckl('dataset_20newsgroups_3_cat.pckl', '/content/drive/MyDrive/NLP_M3_data')

In [ ]:
tra_data = dataset_20newsgroups_3_cat['tra_data']
tra_labels = dataset_20newsgroups_3_cat['tra_labels']
val_data = dataset_20newsgroups_3_cat['val_data']
val_labels = dataset_20newsgroups_3_cat['val_labels']

In [ ]:
len(tra_data), len(tra_labels), len(val_data), len(val_labels)

In [ ]:
tra_data[0].shape

## 5. LSTM Training:

In [ ]:
import pickle
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt

In [ ]:
def load_pckl(file_name, path=None):
    if path is not None:
        file_name = os.path.join(path, file_name)

    with open(file_name, 'rb') as f:
        data = pickle.load(f)
    return data


def save_pckl(d, file_name, pr=None, path=None):
    if path is not None:
        file_name = os.path.join(path, file_name)

    with open(file_name, 'wb') as f:
        pickle.dump(d, f, protocol=pr if pr is not None else pickle.DEFAULT_PROTOCOL)

In [ ]:
ds_name = 'dataset_20newsgroups_3_cat.pckl'


# Load the embedded 20 Newsgroups dataset
dataset = load_pckl(ds_name,
                    '/content/drive/MyDrive/NLP_M3_data')
tra_data = dataset['tra_data']
tra_labels = dataset['tra_labels']
val_data = dataset['val_data']
val_labels = dataset['val_labels']

ok_idx = [i for i, s in enumerate(tra_data) if len(s)>0]
tra_data = [tra_data[i] for i in ok_idx]
tra_labels = [tra_labels[i] for i in ok_idx]

ok_idx = [i for i, s in enumerate(val_data) if len(s)>0]
val_data = [val_data[i] for i in ok_idx]
val_labels = [val_labels[i] for i in ok_idx]

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
batch_size = 50

# make preprocessing function converting data to torch tensors
def preprocess(data, labels):
    # Make random crops of the sequences up to max_len length,
    # Pad short sequences to ensure consistent sequence length
    max_len = 256  # Maximum sequence length
    lens = [len(d) for d in data]  # Get the length of each data point

    # Calculate random offsets for padding
    ofs = [np.random.randint(0, max(1, len_i - max_len)) for len_i in lens]

    # Apply padding based on offsets
    data = [d[o:o + max_len] for d, o in zip(data, ofs)]
    max_len = max([len(d) for d in data])  # Update maximum sequence length after padding

    # Pad sequences with zeros to match the maximum sequence length
    # pad_width contains size of padding  left and righ in each dimension
    data_padded = [np.pad(d, pad_width=((0, max_len - len(d)),
                                        (0, 0))) for d in data]

    # Convert data and labels to NumPy arrays
    data_padded = np.array(data_padded)
    labels = np.array(labels)

    # Convert NumPy arrays to PyTorch tensors
    # and move them to the specified device (e.g., GPU)
    data_t = torch.tensor(data_padded, dtype=torch.float32).to(device)
    labels_t = torch.tensor(labels, dtype=torch.int64).to(device)

    return data_t, labels_t

# make data loader

train_loader = DataLoader(list(zip(tra_data, tra_labels)),
                          batch_size=batch_size, shuffle=True,
                          collate_fn=lambda x: preprocess(*zip(*x)))

test_loader = DataLoader(list(zip(val_data, val_labels)),
                         batch_size=batch_size, shuffle=True,
                         collate_fn=lambda x: preprocess(*zip(*x)))

In [ ]:
class ClassificationRNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(ClassificationRNN, self).__init__()
        self.embedding = nn.Linear(input_dim, embedding_dim)

        if type(hidden_dim) == int:
          hidden_dim = [hidden_dim]

        self.rnn = []
        for i, hidd_d in enumerate(hidden_dim):
          prev_d = embedding_dim if i == 0 else hidden_dim[i-1]
          rnn = nn.LSTM(prev_d, hidd_d, batch_first=True)
          self.add_module(f'lstm_{i}', rnn)
          self.rnn.append(rnn)

        self.fc = nn.Linear(hidd_d, output_dim)

    def forward(self, x):
        embedded = F.relu(self.embedding(x))
        for rnn in self.rnn:
          embedded, (hidden_hn, hidden_cn) = rnn(embedded)

        output = embedded
        rnn_out = hidden_hn[0]  # hn
        return self.fc(rnn_out)

In [ ]:
# Initialize the model
input_dim = tra_data[0].shape[1]
embedding_dim = 100
hidden_dim = 256  # you can specify a list of number of units in sequential LSTM layers
output_dim = 3

model = ClassificationRNN(input_dim, embedding_dim, hidden_dim, output_dim).to(device)

# Define the sparse cross-entropy loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters())

# Train the model
n_epochs = 180

tra_loss_hist = []
val_loss_hist = []
val_acc_hist = []

for epoch in range(n_epochs):
    train_loss = 0.
    valid_loss = 0.

    model.train()
    for batch in train_loader:
        data, labels = batch
        optimizer.zero_grad()
        output = model(data)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    tra_loss_hist.append(train_loss)

    model.eval()
    with torch.no_grad():
        correct = []
        for batch in test_loader:
            data, labels = batch
            output = model(data)
            loss = criterion(output, labels)

            valid_loss += loss.item()

            pred_class = torch.argmax(output, dim=1)
            corr = pred_class == labels
            correct.append(corr.detach().cpu().numpy())

        valid_loss /= len(test_loader)
        correct = np.concatenate(correct)
        accuracy = np.mean(correct)
        print(f"{epoch}:\t Test loss: {valid_loss}; accuracy: {accuracy}")

        val_loss_hist.append(valid_loss)
        val_acc_hist.append(accuracy)

In [ ]:
# plot loss and accuracy on 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(tra_loss_hist, label='train')
ax1.plot(val_loss_hist, label='test')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.legend()

ax2.plot(val_acc_hist)
ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')
plt.show()

print(f'best validation accuracy: {np.max(val_acc_hist)} @ epoch {np.argmax(val_acc_hist)}')

## 6. Exercise 1.

Work in 3 groups, use the 3 datasets.
Optimize the model architecture: # layers, units per layer (hidden_dim) and the embedding_dim to improve validation accuracy.

## 7. Exercise 2.

Use your pretrained model from previous modules for embedding generation. Hint - you can use same Flair interface as for Glove and Roberta, just loading your model. E.g., given your model file 'my_model.pt', you can create document embedder:

```
my_model = torch.load('my_model.pt')
document_embeddings_my_model = DocumentPoolEmbeddings([my_model])
```

alternatively - you can combine the Glove embeddings and your emebedding, or the roberta ones:
```
document_embeddings_stacked = DocumentPoolEmbeddings([my_model, glove_embedding])
```

# Thu evening (Transfer learning for PoS tagging/NER)

### 0. Libs and utils

In [ ]:
!pip install transformers[torch] datasets evaluate seqeval

In [ ]:
from datasets import load_dataset
import datasets

from transformers import AutoTokenizer

from transformers import AutoModelForTokenClassification, AutoModel
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import numpy as np
import matplotlib.pyplot as plt
import evaluate

import torch.cuda
import torch

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

### 1. load dataset

Dataset info: https://huggingface.co/datasets/conll2003

In [ ]:
dataset = load_dataset("conll2003", num_proc=4)

In [ ]:
len(dataset["train"]), len(dataset["test"])

In [ ]:
dataset.keys()

In [ ]:
ds_tra = dataset['train']

In [ ]:
ds_tra

In [ ]:
ds_tra.column_names

In [ ]:
ds_tra[0]

https://cs.nyu.edu/~grishman/jet/guide/PennPOS.html

In [ ]:
pos_tags_info = dataset['train'].features['pos_tags'].feature
class_names = pos_tags_info.names

class_idx_to_class_name = dict(enumerate(class_names))
class_name_to_class_idx = {v:k for k, v in class_idx_to_class_name.items()}
n_classes = len(class_names)

class_types = list(set(class_names))  # get unique

class_idx_to_class_name

### 2. Load tokenizer

In the transformer library for most models you have the associated tokenizer.

The methods AutoTokenizer and AutoModel instantiate a model of proper type.

In [ ]:
mod_name = 'bert-base-uncased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(mod_name)

In [ ]:
tokens = tokenizer.tokenize("Decoding is going the other way around: from vocabulary indices, we want to get a string. This can be done with the decode() method as follows")
print(tokens)

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

In [ ]:
decoded = tokenizer.decode(token_ids)
print(decoded)

### 3. Load model

In [ ]:
model = AutoModel.from_pretrained(mod_name)

In [ ]:
example = dataset["test"][560]
example_txt = example['tokens']
print(example_txt)

In [ ]:
example_tokens_ids = tokenizer(example_txt, is_split_into_words=True, return_tensors='pt')
example_tokens_ids

In [ ]:
np_ids = example_tokens_ids['input_ids'].numpy()[0]
tokens_r = tokenizer.convert_ids_to_tokens(np_ids)
text_r = tokenizer.convert_tokens_to_string(tokens_r)
text_r
#tokens_r

In [ ]:
res = model(**example_tokens_ids)

In [ ]:
for k, v in res.items():
  print(k, v.shape)

So.. What kind of model is this 'bert-base-uncased'?

Lets use it for transfer learning - we use it's features to do token classification. For this we create a model of type `AutoModelForTokenClassification`:

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(mod_name, num_labels=n_classes).to(device)

In [ ]:
res = model(**example_tokens_ids.to(device))

In [ ]:
for k, v in res.items():
  print(k, v.shape)

### 4. Prepare dataset

we can use the tokenizer to covert text into token-ids. but there will be more of them than words. Our lables - are word based. We need thus to generate proper label - per token:

In [ ]:
def tokenize_ner(examples, print_info=False):
  # Extract word tokens and NER labels from examples
  batch_words = examples['tokens']  # these are word tokens
  batch_ner_tags = examples['pos_tags']  # examples['ner_tags']  # these are NER-labels per word

  batch_tokens = tokenizer(batch_words, is_split_into_words=True, truncation=True)
  batch_labels = []

  LBL_IGN = -100  # label for tokens to be ignored during training

  # Iterate through each example in the batch
  for sample_idx, sample_ner_tags in enumerate(batch_ner_tags):
    sample_labels = []  # To store NER labels for tokens in the current example

    word_idxs = batch_tokens.word_ids(batch_index=sample_idx)
    if print_info:
      sample_token_ids = batch_tokens['input_ids'][sample_idx]
      print(word_idxs, sample_ner_tags, tokenizer.convert_ids_to_tokens(sample_token_ids))

    for word_idx in word_idxs:
      if word_idx is None:  # Ignore tokens that are not related to real words
        sample_labels.append(LBL_IGN)
      else:
        token_label = sample_ner_tags[word_idx]
        sample_labels.append(token_label)  # Store the NER label for the token

    # Add the NER labels for the current example to the batch_labels list
    batch_labels.append(sample_labels)

  # Add the batch_labels to the tokenized batch
  batch_tokens['labels'] = batch_labels

  return batch_tokens

Lets inspect what and how does it do:

In [ ]:
res = tokenize_ner(dataset['train'][560:564], print_info=True)
print(res)

In [ ]:
for token, token_id, label in zip(tokenizer.convert_ids_to_tokens(res['input_ids'][0]), res['input_ids'][0], res['labels'][0]):
  print(f'{token:<15} {token_id:<10} {label}')

In [ ]:
tokenized_datasets = dataset.map(tokenize_ner, batched=True, num_proc=4)

In [ ]:
tokenized_datasets['train'][560]

In [ ]:
tokenized_datasets['train'][0].keys()

In [ ]:
res = model(input_ids=torch.tensor(tokenized_datasets['train'][560:561]['input_ids']).to(device))

In [ ]:
for k, v in res.items():
  print(k, v.shape)

Lastly we need a collator.

It ensures that tokenized inputs, labels, and other relevant data are properly formatted and batched together during training. This is particularly useful for tasks like Named Entity Recognition (NER) and part-of-speech tagging where token-level labels need to be aligned with input tokens.

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

### 5. Performnce evaluation

In [ ]:
metric = evaluate.load("poseval")  # seqeval for NER

In [ ]:
# Define a function to compute evaluation metrics for a given batch of predictions and labels.
# The function takes eval_pred as input, which contains batch_logits and batch_labels.
def compute_metrics(eval_pred, print_info=False):
    # Extract batch_logits and batch_labels from eval_pred.
    batch_logits, batch_labels = eval_pred

    # Compute batch_predictions by selecting the class with the highest probability.
    batch_predictions = np.argmax(batch_logits, axis=-1)

    # Initialize lists to store filtered predictions and labels for each sample in the batch.
    filtered_hr_batch_predictions = []
    filtered_hr_batch_labels = []

    # Iterate over samples in the batch to filter out padding tokens (-100).
    for sample_prediction, sample_label in zip(batch_predictions, batch_labels):
        filtered_hr_sample_predictions = []
        filtered_hr_sample_labels = []

        # Iterate over predictions and labels in each sample.
        for prediction, label in zip(sample_prediction, sample_label):
            # Check if the label is not a padding token (-100).
            if label != -100:
                # Convert prediction and label indices to class names using class_idx_to_class_name mapping.
                filtered_hr_sample_predictions.append(class_idx_to_class_name[prediction])
                filtered_hr_sample_labels.append(class_idx_to_class_name[label])

        # Append filtered predictions and labels for the sample to the batch lists.
        filtered_hr_batch_predictions.append(filtered_hr_sample_predictions)
        filtered_hr_batch_labels.append(filtered_hr_sample_labels)

    # Compute evaluation metrics using the filtered predictions and labels.
    metric_res = metric.compute(predictions=filtered_hr_batch_predictions,
                                references=filtered_hr_batch_labels)

    # Optionally print information about filtered predictions, labels, and metric results.
    if print_info:
        print(filtered_hr_batch_predictions)
        print(filtered_hr_batch_labels)
        print(metric_res)

    # Create a dictionary to store computed metrics.
    all_metrics = {k: v for k, v in metric_res.items() if type(v) is not dict}
    all_metrics = {**all_metrics, **metric_res['weighted avg']}

    # Compute and add F1 scores for specific class types to the metrics dictionary.
    for ct in class_types:
        v = metric_res.get(ct, None)
        all_metrics[ct + '_f1'] = v['f1-score'] if v is not None else 0.

    # Return the computed evaluation metrics.
    return all_metrics

In [ ]:
ds_test = tokenized_datasets['test']

res = model(input_ids=torch.tensor(ds_test['input_ids'][:1]).to(device),
            attention_mask=torch.tensor(ds_test['attention_mask'][:1]).to(device)
            )

In [ ]:
# don't forget to copy data to cpu-accessibel memory and convert to NumPy
pred = res.logits.detach().cpu().numpy()
lbl = tokenized_datasets['test']['labels'][:1]

In [ ]:
metrics_res = compute_metrics((pred, lbl), print_info=True)

In [ ]:
metrics_res

Since the last model layer doing the 47-way classification is not yet trained - the model spits out rubbish, and the scores are 0

### 6. Training

In [ ]:
# rm -rf ner logs_ner

In [ ]:
rm -rf pos logs_pos

In [ ]:
training_args = TrainingArguments(output_dir="pos",
                                  num_train_epochs=10,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",

                                  per_device_train_batch_size=16,  # batch size per device during training
                                  per_device_eval_batch_size=16,   # batch size for evaluation
                                  warmup_steps=250,                # number of warmup steps for learning rate scheduler
                                  weight_decay=0.01,               # strength of weight decay
                                  logging_dir='./logs_pos',        # directory for storing logs
                                  logging_steps=10,
                                  # optim="adafactor"
                                  )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [ ]:
!ls

In [ ]:
trainer.train()

In [ ]:
def plot_hist(log_hist):
  """
  Helper function to aggregate and visualize training history
  from trainers's logs
  """
  last_loss = 0
  s = 4
  sfx = 'eval_'
  e_loss_name = sfx+'loss'
  sfx_skip = ['_per_second', 'runtime', 'epoch', 'step', 'confusion_matrix']

  steps = []
  loss = []

  e_steps = []
  e_loss = []
  e_mtr = {}

  e_cms = []

  for el in log_hist:
    if e_loss_name in el:
      for k, v in el.items():
        if any([s in k for s in sfx_skip]):
          continue


        if k == e_loss_name:
          e_loss.append(v)
        else:
          if k not in e_mtr:
            e_mtr[k] = []
          e_mtr[k].append(v)

      e_steps.append(el['step'])
      if 'confusion_matrix' in el:
        e_cms.append(el['confusion_matrix'])

    else:
      if 'loss' in el:
        steps.append(el['step'])
        loss.append(el['loss'])

  n_fig = len(e_mtr)+1

  fix, ax = plt.subplots(1, n_fig, figsize=(s*n_fig, s*1))
  ax[0].plot(steps, loss, alpha=0.5, label='tra')
  ax[0].plot(e_steps, e_loss, alpha=0.5, label='val')
  ax[0].set_xlabel('steps')
  ax[0].set_title('loss')
  ax[0].legend()

  for idx, (lbl, vals) in enumerate(e_mtr.items()):
    i = idx+1

    #print(lbl, vals, e_steps)
    ax[i].plot(e_steps, vals, alpha=0.5, label='val')
    ax[i].set_xlabel('steps')
    ax[i].set_title(lbl)

    #ax[i].legend()

  plt.show()
  plt.close()

  for i, cm in enumerate(e_cms):
    plt.matshow(cm, cmap=plt.cm.Blues, alpha=0.3)
    plt.title(e_steps[i])
    plt.show()

In [ ]:
plot_hist(trainer.state.log_history)

### 7. Exercise

Try modifying the code to do NER tag identification